In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.preprocessing import LabelEncoder

In [2]:
path = "/Users/thrilok/Desktop/mantra_collab_job/work_files/latest_broko_code"
df = pd.read_csv(path  + "/Dataset/ML_CLEAN_DATA__Bdv2.4_RES.csv")
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181512 entries, 0 to 181511
Data columns (total 25 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ML_Number       181490 non-null  object 
 1   Area            181490 non-null  object 
 2   Municipality    178872 non-null  object 
 3   Community       178872 non-null  object 
 4   Postal_Code     181359 non-null  object 
 5   Postal_Short    181359 non-null  object 
 6   Style           181490 non-null  object 
 7   Type            181490 non-null  object 
 8   Cluster         181490 non-null  object 
 9   List_Price      181485 non-null  float64
 10  Cluster_Price   181490 non-null  float64
 11  Taxes           181488 non-null  float64
 12  Cluster_Tax     181490 non-null  float64
 13  Bedrooms        181490 non-null  float64
 14  Washrooms       181490 non-null  float64
 15  Basement1       181490 non-null  object 
 16  Days_On_Market  181490 non-null  float64
 17  Exterior1 

In [3]:
df = df[df['Municipality'] == 'Mississauga']
df = df[df['Community'] == 'lakeview']
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 420 entries, 107104 to 108007
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ML_Number       420 non-null    object 
 1   Area            420 non-null    object 
 2   Municipality    420 non-null    object 
 3   Community       420 non-null    object 
 4   Postal_Code     420 non-null    object 
 5   Postal_Short    420 non-null    object 
 6   Style           420 non-null    object 
 7   Type            420 non-null    object 
 8   Cluster         420 non-null    object 
 9   List_Price      420 non-null    float64
 10  Cluster_Price   420 non-null    float64
 11  Taxes           420 non-null    float64
 12  Cluster_Tax     420 non-null    float64
 13  Bedrooms        420 non-null    float64
 14  Washrooms       420 non-null    float64
 15  Basement1       420 non-null    object 
 16  Days_On_Market  420 non-null    float64
 17  Exterior1       420 non-null    

In [4]:
drop_cols = ["ML_Number","Postal_Code","Sold_Price","Month_Year","HPI_for_Month","Community","Municipality","Area","Cluster","Cluster_Price","Cluster_Tax","List_Price","Days_On_Market","lat","lng"]
# drop_cols = ['ML_Number','Sold_Date',"Address","Area","Postal_Code","Air_Conditioning","Exterior1","Rooms","Tax_Year","Water_Included","property_type"]#,"lat","lng"]
data = df.drop(drop_cols, axis=1)
data.isnull().sum()

Postal_Short       0
Style              0
Type               0
Taxes              0
Bedrooms           0
Washrooms          0
Basement1          0
Exterior1          0
Garage_Type       60
HPI_Sold_Price     0
dtype: int64

In [5]:
data = data.dropna(axis = 0, how ='any') 

In [6]:
q1 = data['HPI_Sold_Price'].quantile(0.25)
q3 = data['HPI_Sold_Price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
lower_bound_outliers = data[data['HPI_Sold_Price'] < lower_bound] 
upper_bound_outliers = data[data['HPI_Sold_Price'] > upper_bound]
# lower_bound_outliers.to_csv(path + '/Dataset/lower_bound_outliers.csv')
# upper_bound_outliers.to_csv(path +'/Dataset/upper_bound_outliers.csv')    
dataset = data[(data['HPI_Sold_Price'] >= lower_bound) & (data['HPI_Sold_Price'] <= upper_bound)]

In [7]:
dataset.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 107104 to 108007
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Postal_Short    340 non-null    object 
 1   Style           340 non-null    object 
 2   Type            340 non-null    object 
 3   Taxes           340 non-null    float64
 4   Bedrooms        340 non-null    float64
 5   Washrooms       340 non-null    float64
 6   Basement1       340 non-null    object 
 7   Exterior1       340 non-null    object 
 8   Garage_Type     340 non-null    object 
 9   HPI_Sold_Price  340 non-null    float64
dtypes: float64(4), object(6)
memory usage: 29.2+ KB


In [8]:
X = dataset.drop('HPI_Sold_Price',axis=1)
y = dataset['HPI_Sold_Price']

In [9]:
# categorical features
categorical_features = [column for column, dtype in X.dtypes.items() if dtype==object]


In [10]:
best_params = pd.read_csv(path + "/Dataset/best_xgb.csv")
best_params

,max_depth,learning_rate,n_estimators,reg_lambda,reg_alpha
0,6,0.265023,49,0.885813,0.089943


In [11]:
params = {
    "max_depth": best_params["max_depth"][0],
    "learning_rate": best_params["learning_rate"][0],
    "n_estimators": best_params["n_estimators"][0],
    "reg_lambda": best_params["reg_lambda"][0],
    "reg_alpha": best_params["reg_alpha"][0],
    # "booster":'gblinear'
}

model = XGBRegressor(**params)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Encode categorical features before training
label_encoders = {}
for feature in categorical_features:
    le = LabelEncoder()
    X_train[feature] = le.fit_transform(X_train[feature])
    X_test[feature] = le.fit_transform(X_test[feature])
    label_encoders[feature] = le


# model.fit(
#     X_train,y_train,
#     eval_set=[(X_test,y_test)],
#     verbose=False
# )

y_pred = model.predict(X_test)

MAPE(y_test, y_pred)

0.1311572033263533

In [20]:
y_pred = model.predict(X_test[:1])
y_pred

array([1708528.69449457])

In [12]:
# model.feature_importances_

In [14]:
# Get the coefficients
coefficients = model.coef_

print(f"intercept: {model.intercept_}")

for feature,coefficients in zip(X.columns,coefficients):
    print(f"{feature} = {coefficients.astype(int)}")

intercept: 794657.4624137628
Postal_Short = 1857
Style = 7614
Type = -66687
Taxes = 12
Bedrooms = 36684
Washrooms = 186934
Basement1 = 1007
Exterior1 = 29059
Garage_Type = -73078


In [26]:
X_test[:1]

a = le.inverse_transform(X_test[:1])

ValueError: y should be a 1d array, got an array of shape (1, 9) instead.

In [18]:
y_pred = model.intercept_ + model.coef_ * X_test[:1]
y_pred

,Postal_Short,Style,Type,Taxes,Bedrooms,Washrooms,Basement1,Exterior1,Garage_Type
107927,796514.691642,802271.76071,727970.145377,886225.562861,941396.535007,1.542396e+06,794657.462414,852777.188196,721579.093905


In [21]:
796514.691642+ 802271.76071+727970.145377+886225.562861+941396.535007+1542396+794657.462414+ 852777.188196 +721579.093905

8065788.440111999

In [ ]:
# from matplotlib import pyplot

# pyplot.bar(range(len(model.feature_importances_)), model.feature_importances_)
# pyplot.show()

AttributeError: 'LinearRegression' object has no attribute 'feature_importances_'

In [ ]:
# for feature,weights in zip(X.columns,model.feature_importances_):
#     print(f"{feature} = {weights *100 }")

In [ ]:
sum(model.feature_importances_)

1.0000000058716978

In [ ]:
Acc = (abs(1 - (abs(y_test- y_pred)/y_test)) * 100).astype(int)

In [ ]:
Acc

92843     90
66180     76
53337     85
31139     99
75379     89
          ..
160861    77
23273     62
134548    65
160847    69
38365     59
Name: HPI_Sold_Price, Length: 28940, dtype: int64

In [ ]:
for col, le in label_encoders.items():
    X_test[col] = le.inverse_transform(X_test[col])

In [ ]:
predictions = X_test
predictions['ML_Number'] = df['ML_Number']
predictions['Month_Year'] = df["Month_Year"]
predictions['HPI_for_Month'] = df["HPI_for_Month"]
predictions['Sold_Price'] = df['Sold_Price']
predictions['HPI_Sold_Price'] = df['HPI_Sold_Price']
predictions['Predicted_HPI_Price']  = y_pred.astype(int)
predictions['Diff']  = abs(predictions['HPI_Sold_Price'] - predictions['Predicted_HPI_Price']).astype(int)
predictions['Accuracy'] = (abs(1 - (abs(predictions['HPI_Sold_Price'] - predictions['Predicted_HPI_Price']))/predictions['HPI_Sold_Price']) * 100).astype(int)
predictions

,Postal_Short,Style,Type,Taxes,Bedrooms,Washrooms,Basement1,Exterior1,Garage_Type,lat,lng,ML_Number,Month_Year,HPI_for_Month,Sold_Price,HPI_Sold_Price,Predicted_HPI_Price,Diff,Accuracy
92843,l3y,Backsplit,Detached,5166.00,4.0,3.0,Finished,Brick,Attached,44.048614,-79.483066,N5544766,Jun 2022,360.3,1325000.0,1.295212e+06,1173882,121330,90
66180,l1s,2-storey,Detached,5456.48,4.0,4.0,Partly Finished,Brick,Built-In,43.855112,-79.039240,E5642717,Jun 2022,360.3,1005000.0,9.824063e+05,1213384,230977,76
53337,l7e,2-storey,Detached,6352.00,4.0,4.0,Finished Walkout,Alum Siding,Built-In,43.906994,-79.750349,W5541105,Mar 2022,391.9,1650000.0,1.482853e+06,1265586,217266,85
31139,n2p,2-storey,Detached,4800.00,4.0,4.0,Finished,Brick,Attached,43.373305,-80.436898,X5442233,Dec 2021,364.0,1375000.0,1.330426e+06,1338644,8218,99
75379,l6x,2-storey,Detached,6673.14,5.0,5.0,Finished,Brick,Built-In,43.669202,-79.816608,W5572960,May 2022,371.3,1671000.0,1.585042e+06,1415441,169601,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160861,m8z,bungalow,Detached,5434.63,3.0,2.0,Finished,Brick,Attached,43.634925,-79.513405,W5902751,Feb 2023,320.2,1250000.0,1.374922e+06,1060602,314319,77
23273,l9s,bungalow,Detached,3774.78,1.0,0.0,Finished,Brick,Other,44.322990,-79.543200,N6311776,Sep 2021,332.3,930000.0,9.856937e+05,618114,367579,62
134548,m6g,2-storey,Semi-Detached,5415.67,3.0,2.0,Finished,Brick,Detached,43.660757,-79.420198,C5873036,Mar 2023,322.4,1400000.0,1.529404e+06,1004747,524657,65
160847,m8y,Backsplit,Detached,5291.37,4.0,3.0,Finished,Brick,Attached,43.632124,-79.496725,W5409096,Nov 2021,355.5,1788020.0,1.771422e+06,1228933,542489,69


In [ ]:
predictions['Accuracy'].sum()/len(predictions['Accuracy'])

81.28645473393227

In [ ]:
# # save the dataframe as a csv file
# predictions.to_csv(path + "/Dataset/21-09-2023/Preds_res_xgbv2.4lplakeview.csv",index=False)

In [ ]:
# import pickle

# pickle.dump(model, open(path + '/models/xgb.pkl', 'wb'))